In [2]:
pip install pandas "pyspark[connect]"


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyspark, sys
print("pyspark:", pyspark.__version__, "python:", sys.version)

pyspark: 4.0.1 python: 3.10.12 (main, Nov 21 2025, 16:07:24) [Clang 17.0.0 (clang-1700.0.13.5)]


In [3]:
!pip install /Users/chenguo/src/chen/spark-streaming/word-count-sc/dist/wordcount_sc-0.1.0-py3-none-any.whl

Processing /Users/chenguo/src/chen/spark-streaming/word-count-sc/dist/wordcount_sc-0.1.0-py3-none-any.whl

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
from pyspark.sql import SparkSession

# Create a Spark session using Connect
spark = (
    SparkSession.builder
    # May need to do `sudo sh -c 'echo "127.0.0.1 spark-connect-svc.spark-operator.svc.cluster.local" >> /etc/hosts'`
    # For kubectl port-forward, k8s sets up a local socket on 127.0.0.1:15002 that tunnels traffic to the service spark-connect-svc 
    # inside the cluster. However, Spark Connect isn’t plain TCP; it’s gRPC/HTTP 2 and often TLS-aware (even if you didn’t configure full HTTPS).
    # During connection setup, the client identifies itself using the authority/hostname portion of the URI (sc://...).
    # If your notebook connects with `.remote("sc://127.0.0.1:15002")`, the client says “authority = 127.0.0.1”, but the server thinks 
    # its own name is `spark-connect-svc.spark-operator.svc.cluster.local`

    # .remote("sc://spark-connect-svc.spark-operator.svc.cluster.local:15002")
    .remote("sc://localhost:15002")
    .appName("LocalJupyterNotebook") # Not very useful for Spark Connect scenario
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true")
    .config("spark.dynamicAllocation.initialExecutors", "1")
    .config("spark.dynamicAllocation.minExecutors", "1")
    .config("spark.dynamicAllocation.maxExecutors", "8")
    .config("spark.default.parallelism", "1024")
    .config("spark.sql.shuffle.partitions", "1024")
    .getOrCreate()
)

spark_app_name = spark.conf.get("spark.app.name")
# Test the connection
print(f"Spark version: {spark.version}")
print(f"Spark Connect session id: {spark.client._session_id}")
print(f"App Name: {spark_app_name}")

Spark version: 4.0.0
Spark Connect session id: 3282c46c-6253-48a5-b23a-5692be3e7129
App Name: LocalJupyterNotebook


/Users/chenguo/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pyspark/sql/connect/conf.py:64: UserWarning: Failed to set spark.dynamicAllocation.enabled to Some(true) due to [CANNOT_MODIFY_CONFIG] Cannot modify the value of the Spark config: "spark.dynamicAllocation.enabled".
See also 'https://spark.apache.org/docs/latest/sql-migration-guide.html#ddl-statements'. SQLSTATE: 46110
  warnings.warn(warn)
/Users/chenguo/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pyspark/sql/connect/conf.py:64: UserWarning: Failed to set spark.dynamicAllocation.shuffleTracking.enabled to Some(true) due to [CANNOT_MODIFY_CONFIG] Cannot modify the value of the Spark config: "spark.dynamicAllocation.shuffleTracking.enabled".
See also 'https://spark.apache.org/docs/latest/sql-migration-guide.html#ddl-statements'. SQLSTATE: 46110
  warnings.warn(warn)
/Users/chenguo/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pyspark/sql/connect/conf.py:64: UserWarning: Failed to set spark.dynamicAllocati

In [7]:
from word_count import WordCountSparkConnectIDE

WordCountSparkConnectIDE.main(spark)

Row(word='_apt:x:100:65534::/nonexistent:/usr/sbin/nologin', count=1)
Row(word='www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin', count=1)
Row(word='Manager:/var/list:/usr/sbin/nologin', count=1)
Row(word='sys:x:3:3:sys:/dev:/usr/sbin/nologin', count=1)
Row(word='(admin):/var/lib/gnats:/usr/sbin/nologin', count=1)
Row(word='daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin', count=1)
Row(word='irc:x:39:39:ircd:/run/ircd:/usr/sbin/nologin', count=1)
Row(word='gnats:x:41:41:Gnats', count=1)
Row(word='System', count=1)
Row(word='man:x:6:12:man:/var/cache/man:/usr/sbin/nologin', count=1)


In [10]:
from word_count import WordCountSparkConnectIDE2

WordCountSparkConnectIDE2.main(spark)

Row(word='_apt:x:100:65534::/nonexistent:/usr/sbin/nologin', count=1)
Row(word='www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin', count=1)
Row(word='Manager:/var/list:/usr/sbin/nologin', count=1)
Row(word='sys:x:3:3:sys:/dev:/usr/sbin/nologin', count=1)
Row(word='(admin):/var/lib/gnats:/usr/sbin/nologin', count=1)
Row(word='daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin', count=1)
Row(word='irc:x:39:39:ircd:/run/ircd:/usr/sbin/nologin', count=1)
Row(word='gnats:x:41:41:Gnats', count=1)
Row(word='System', count=1)
Row(word='man:x:6:12:man:/var/cache/man:/usr/sbin/nologin', count=1)


In [8]:
print(spark.conf.get("spark.dynamicAllocation.enabled", "not set"))
print(spark.conf.get("spark.dynamicAllocation.shuffleTracking.enabled", "not set"))
print(spark.conf.get("spark.dynamicAllocation.initialExecutors", "not set"))
print(spark.conf.get("spark.dynamicAllocation.minExecutors", "not set"))
print(spark.conf.get("spark.dynamicAllocation.maxExecutors", "not set"))
print(spark.conf.get("spark.default.parallelism", "not set"))
print(spark.conf.get("spark.sql.shuffle.partitions", "not set"))

true
true
1
1
50
1024
1024


In [9]:
df = spark.range(3)
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



In [34]:
from pyspark.sql.functions import rand
df = spark.range(0, 50_000_000, numPartitions=2000).withColumn("x", rand())
a  = df.repartition(2000, "id").groupBy((df.id % 500).alias("k")).count()
b  = df.repartition(2000, "id").groupBy((df.id % 500).alias("k")).avg("x")
a.join(b, "k").count()


KeyboardInterrupt


KeyboardInterrupt



In [9]:
spark.stop()